In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import pandas as pd
import os

# # Setup WebDriver
driverOptions = webdriver.ChromeOptions()
driverOptions.browser_version = "120"
driver = webdriver.Chrome(options=driverOptions)


#Scrape the table data
table_data = []
# Create DataFrame for the scraped data
df = pd.DataFrame()  
#Create headers
headers = []
#add all the pages urls
pages = []
for page_number in range(1, 4):
        url_start = 'https://finance.yahoo.com/crypto?count=100&offset='
        url = url_start + str(page_number*100)
        pages.append(url)
        print(url)

# print(pages)
# Loop through the pages 
for page in pages : 
    # Open the webpage
    driver.get(page)
    driver.delete_all_cookies()
    
    #find if scrollable element is there or not
    try:
        scroll = driver.find_element(By.ID, 'scroll-down-btn')
        if scroll:
            driver.execute_script("document.getElementsByClassName('scroll-down-wrapper')[0].remove()")
            acceptButton = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "button")))
            acceptButton.click()
            # print(acceptButton)
    except:
            print('No element')
        
        # print(scroll.get_attribute('outerHTML'))
    
    table = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "table")))
    # print(table)
            
    # Store the window handle of the original window so we can switch back to it later
    original_window = driver.current_window_handle

    # Extract the headers
    headers = [th.text for th in table.find_elements(By.TAG_NAME, "th")]

    # # Iterate through each row in the table
    counter = 0    
    for row in table.find_elements(By.TAG_NAME, "tr"):
            if counter <= 2: #check if the counter has reached greater than 10
                dataCols = row.find_elements(By.TAG_NAME, "td")
                if dataCols:
                    lastCol = dataCols[-1]
                    # ele = lastCol.find_element(By.TAG_NAME, 'a')
                    ele = WebDriverWait(lastCol, 20).until(EC.element_to_be_clickable((By.TAG_NAME, 'a')) )
                    href = ele.get_attribute('href')
                    if href : 
                        driver.execute_script("window.open(arguments[0]);", href)
                        # Wait for the new window/tab to appear
                        WebDriverWait(driver, 20).until(EC.number_of_windows_to_be(2))    
                        # Iterate through the open windows, switch to the new one when found
                        for window_handle in driver.window_handles:
                            if window_handle != original_window:
                                driver.switch_to.window(window_handle)
                                break
                        
                        try:
                            scroll = driver.find_element(By.ID, 'scroll-down-btn')
                            if scroll:
                                driver.execute_script("document.getElementsByClassName('scroll-down-wrapper')[0].remove()")
                                acceptButton = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "button")))
                                acceptButton.click()
                                # print(acceptButton)
                        except:
                            print('No element')
                        #Xpath for the share button
                        xpath = "//div[contains(concat(' ', normalize-space(@class), ' '), ' D(ib) ') and contains(concat(' ', normalize-space(@class), ' '), ' Bdrs(3px) ') and contains(concat(' ', normalize-space(@class), ' '), ' Mend(10px)--tab1024 ')]//button"
                        action_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, xpath)))
                        # print(action_button.get_attribute('outerHTML'))
                        action_button.click()
                        xpath_share = "//div[contains(concat(' ', normalize-space(@class), ' '), ' P(30px) ') and contains(concat(' ', normalize-space(@class), ' '), ' Pb(35px)! ') and contains(concat(' ', normalize-space(@class), ' '), ' C($tertiaryColor) ')]//button[@data-test='share-dl-button']"
                        download_button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, xpath_share))).click()
                        driver.close()
                        driver.switch_to.window(original_window)
            else:
                #break from the loop when counter is greater than 10
                break
            #Increase the counter
            counter += 1
            

    # Extract text from each cell in the row
    # Scrape the rows
    for row in table.find_elements(By.TAG_NAME, "tr"):
        cols = row.find_elements(By.TAG_NAME, "td")
        table_data.append([col.text for col in cols])
 
print(headers)   
# # # Clean DataFrame to remove empty rows (if any)
# # # Create a DataFrame
df = pd.DataFrame(table_data, columns=headers)
df.dropna(how='all', inplace=True)
# # #Save DataFrame to CSV
df.to_csv('table_data.csv', index=False)
print(df)
            
            
            

https://finance.yahoo.com/crypto?count=100&offset=100
https://finance.yahoo.com/crypto?count=100&offset=200
https://finance.yahoo.com/crypto?count=100&offset=300
No element
No element
No element
No element
No element
No element
['Symbol', 'Name', 'Price (Intraday)', 'Change', '% Change', 'Market Cap', 'Volume in Currency (Since 0:00 UTC)', 'Volume in Currency (24Hr)', 'Total Volume All Currencies (24Hr)', 'Circulating Supply', '52 Week Range', 'Day Chart']
             Symbol                Name Price (Intraday)     Change  \
1          ROSE-USD   Oasis Network USD         0.101000  +0.001100   
2          BONK-USD            Bonk USD         0.000011  -0.000000   
3          KLAY-USD          Klaytn USD         0.192257  -0.001371   
4          PYTH-USD    Pyth Network USD         0.440706  +0.029847   
5           FTT-USD       FTX Token USD           1.9689    -0.6668   
..              ...                 ...              ...        ...   
298         WRX-USD          WazirX USD   